# Querying from Mongo DataBase

In this notebook, we'll make querys to our Mongo database and create a pandas dataframe. 

-----

**First things first, let's import some things...**

In [2]:
import json
import pandas as pd
from pymongo import MongoClient

<br>

**Creating a Mongo Client**

In [3]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client['pbmnet']
curriculos = db.curriculos

<br>

**Querying...**

In [4]:
# curriculos.find_one({"PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.AUTORES.@NOME-PARA-CITACAO":"BONOTTO, E.M."})
d = curriculos.find_one({},{'_id':0,'@NUMERO-IDENTIFICADOR':1,'DADOS-GERAIS.@NOME-COMPLETO': 1,
                            'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@TITULO-DO-ARTIGO': 1,
                           'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@ANO-DO-ARTIGO':1})

In [7]:
d

{'@NUMERO-IDENTIFICADOR': '6115398975075240',
 'DADOS-GERAIS': {'@NOME-COMPLETO': 'Manuel Francisco Zuloeta Jimenez'},
 'PRODUCAO-BIBLIOGRAFICA': {'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': [{'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Weak almost periodic motions, minimality and stability in impulsive semidynamical systems',
      '@ANO-DO-ARTIGO': '2014'}},
    {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'On impulsive semidynamical systems: minimal, recurrent and almost periodic motions',
      '@ANO-DO-ARTIGO': '2014'}},
    {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Negative trajectories in impulsive semidynamical systems',
      '@ANO-DO-ARTIGO': '2015'}},
    {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Convergence for non-autonomous semidynamical systems with impulses',
      '@ANO-DO-ARTIGO': '2019'}}]}}}

In [5]:
d['DADOS-GERAIS']['@NOME-COMPLETO']

'Manuel Francisco Zuloeta Jimenez'

In [17]:
unwind = list(curriculos.aggregate([
                           {'$project':{
        '_id':1,
        '@NUMERO-IDENTIFICADOR':1,
        'DADOS-GERAIS.@NOME-COMPLETO': 1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@NOME-INSTITUICAO-EMPRESA':1,
        'DADOS-GERAIS.ENDERECO.ENDERECO-PROFISSIONAL.@CODIGO-INSTITUICAO-EMPRESA':1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@TITULO-DO-ARTIGO': 1,
        'PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO.DADOS-BASICOS-DO-ARTIGO.@ANO-DO-ARTIGO':1
                                       }},
                            {'$unwind': '$PRODUCAO-BIBLIOGRAFICA.ARTIGOS-PUBLICADOS.ARTIGO-PUBLICADO'},
                          ]))

In [23]:
unwind[2]

{'_id': ObjectId('5d6551dfc0e1f77ce11401ba'),
 '@NUMERO-IDENTIFICADOR': '6115398975075240',
 'DADOS-GERAIS': {'@NOME-COMPLETO': 'Manuel Francisco Zuloeta Jimenez',
  'ENDERECO': {'ENDERECO-PROFISSIONAL': {'@CODIGO-INSTITUICAO-EMPRESA': '198100000000',
    '@NOME-INSTITUICAO-EMPRESA': 'Universidade Tecnológica Federal do Paraná'}}},
 'PRODUCAO-BIBLIOGRAFICA': {'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'DADOS-BASICOS-DO-ARTIGO': {'@TITULO-DO-ARTIGO': 'Negative trajectories in impulsive semidynamical systems',
     '@ANO-DO-ARTIGO': '2015'}}}}}

In [36]:
def Unest_Titulo_Artigo(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DADOS-BASICOS-DO-ARTIGO']['@TITULO-DO-ARTIGO'].strip()

def Unest_Endereco_Nome(cell):    
    return cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@NOME-INSTITUICAO-EMPRESA']

def Unest_Endereco_Codigo(cell):
    return cell['ENDERECO']['ENDERECO-PROFISSIONAL']['@CODIGO-INSTITUICAO-EMPRESA']

def Unest_Ano_Artigo(cell):
    return cell['ARTIGOS-PUBLICADOS']['ARTIGO-PUBLICADO']['DADOS-BASICOS-DO-ARTIGO']['@ANO-DO-ARTIGO'].strip()

In [37]:
pd.DataFrame(unwind).head()

,@NUMERO-IDENTIFICADOR,DADOS-GERAIS,PRODUCAO-BIBLIOGRAFICA,_id
0,6115398975075240,{'@NOME-COMPLETO': 'Manuel Francisco Zuloeta J...,{'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'...,5d6551dfc0e1f77ce11401ba
1,6115398975075240,{'@NOME-COMPLETO': 'Manuel Francisco Zuloeta J...,{'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'...,5d6551dfc0e1f77ce11401ba
2,6115398975075240,{'@NOME-COMPLETO': 'Manuel Francisco Zuloeta J...,{'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'...,5d6551dfc0e1f77ce11401ba
3,6115398975075240,{'@NOME-COMPLETO': 'Manuel Francisco Zuloeta J...,{'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'...,5d6551dfc0e1f77ce11401ba
4,1386464075867571,{'@NOME-COMPLETO': 'Roberto dos Santos Rabello...,{'ARTIGOS-PUBLICADOS': {'ARTIGO-PUBLICADO': {'...,5d6551dfc0e1f77ce11401bb


In [38]:
df = pd.DataFrame(unwind)

df['NOME-COMPLETO']     = df["DADOS-GERAIS"].apply(lambda cell: cell['@NOME-COMPLETO'].strip())
df['NOME-INSTITUICAO']     = df["DADOS-GERAIS"].apply(Unest_Endereco_Nome)
df['CODIGO-INSTITUICAO']     = df["DADOS-GERAIS"].apply(Unest_Endereco_Codigo)
df['TITULO-DO-ARTIGO'] = df["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Titulo_Artigo)
df['ANO-DO-ARTIGO']    = df["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Ano_Artigo)
df['ANO-DO-ARTIGO']    = df["PRODUCAO-BIBLIOGRAFICA"].apply(Unest_Ano_Artigo)

df.drop(labels='PRODUCAO-BIBLIOGRAFICA',axis=1,inplace=True)

KeyError: 'ENDERECO-PROFISSIONAL'

In [21]:
df

,@NUMERO-IDENTIFICADOR,DADOS-GERAIS,_id,TITULO-DO-ARTIGO,ANO-DO-ARTIGO
0,6115398975075240,Manuel Francisco Zuloeta Jimenez,5d6551dfc0e1f77ce11401ba,"Weak almost periodic motions, minimality and s...",2014
1,6115398975075240,Manuel Francisco Zuloeta Jimenez,5d6551dfc0e1f77ce11401ba,"On impulsive semidynamical systems: minimal, r...",2014
2,6115398975075240,Manuel Francisco Zuloeta Jimenez,5d6551dfc0e1f77ce11401ba,Negative trajectories in impulsive semidynamic...,2015
3,6115398975075240,Manuel Francisco Zuloeta Jimenez,5d6551dfc0e1f77ce11401ba,Convergence for non-autonomous semidynamical s...,2019
4,1386464075867571,Roberto dos Santos Rabello,5d6551dfc0e1f77ce11401bb,Monitorando a Comunicação na CV-Muzar com o Us...,2010
5,1386464075867571,Roberto dos Santos Rabello,5d6551dfc0e1f77ce11401bb,Dia-log: análise de marcadores verbais para id...,2008
6,1386464075867571,Roberto dos Santos Rabello,5d6551dfc0e1f77ce11401bb,Um Sistema Multiagente para Identificar Falhas...,2009
7,1386464075867571,Roberto dos Santos Rabello,5d6551dfc0e1f77ce11401bb,O uso de agentes inteligentes no apoio da inte...,2010
8,1386464075867571,Roberto dos Santos Rabello,5d6551dfc0e1f77ce11401bb,Um modelo para a implementação de contratos el...,2012
9,1386464075867571,Roberto dos Santos Rabello,5d6551dfc0e1f77ce11401bb,Interação e Autismo: Uso de Agentes Inteligent...,2011
